# 03 - Geração de Dados de Interação com LLM

Este notebook implementa a **Etapa 3** do pipeline SINKT: geração de 3000-6000 interações simuladas usando LLM para respostas realistas.

## Objetivo
Gerar sequências de interações realistas para cada estudante, incluindo respostas a questões geradas por LLM.

## Saída
- `data/output/interactions.json`: Arquivo JSON contendo todas as interações simuladas com respostas realistas

## Importação de Bibliotecas

In [38]:
import json
import os
from datetime import datetime, timedelta
from typing import Dict, List, Any, Tuple
import numpy as np
import pandas as pd
from collections import defaultdict
import random
import time

# Importar OpenAI
try:
    from openai import OpenAI
    print("✅ OpenAI importado com sucesso")
except ImportError:
    print("❌ OpenAI não instalado. Instale com: pip install openai")
    raise

print("✅ Bibliotecas importadas com sucesso")

✅ OpenAI importado com sucesso
✅ Bibliotecas importadas com sucesso


## Inicializar Cliente OpenAI

In [39]:
# Inicializar cliente OpenAI
# A API key é automaticamente lida de OPENAI_API_KEY
client = OpenAI()

print("✅ Cliente OpenAI inicializado")
print("   Usando modelo: gpt-4.1-mini")
print("   API Key: Carregada de OPENAI_API_KEY")

✅ Cliente OpenAI inicializado
   Usando modelo: gpt-4.1-mini
   API Key: Carregada de OPENAI_API_KEY


## Carregamento de Dados

In [40]:
# Carregar perfis
with open('data/output/notebooks/geracao_perfis/profiles.json', 'r', encoding='utf-8') as f:
    profiles_data = json.load(f)
profiles = profiles_data['profiles']

# Carregar estudantes
with open('data/output/notebooks/geracao_estudantes/students.json', 'r', encoding='utf-8') as f:
    students_data = json.load(f)
students = students_data['students']

# Carregar questões
with open('data/json/questions_graph.json', 'r', encoding='utf-8') as f:
    questions_data = json.load(f)
questions = questions_data.get('questions', [])

# Carregar conceitos
with open('data/json/concepts_graph.json', 'r', encoding='utf-8') as f:
    concepts_data = json.load(f)
concepts = concepts_data.get('concepts', [])

print(f"✅ Dados carregados:")
print(f"  - Perfis: {len(profiles)}")
print(f"  - Estudantes: {len(students)}")
print(f"  - Questões: {len(questions)}")
print(f"  - Conceitos: {len(concepts)}")

✅ Dados carregados:
  - Perfis: 6
  - Estudantes: 100
  - Questões: 680
  - Conceitos: 251


## Configuração de Parâmetros

In [41]:
# Configurações
MIN_INTERACTIONS_PER_STUDENT = 30
MAX_INTERACTIONS_PER_STUDENT = 60
SEED = 42
LLM_MODEL = "gpt-4.1-mini"
LLM_TEMPERATURE = 0.7
LLM_MAX_TOKENS = 200
LLM_TIMEOUT = 10

CHECKPOINT_INTERVAL = 10
CHECKPOINT_FILE = 'data/output/interactions_checkpoint.json'

ERROR_TYPES = [
    'misconception',
    'careless',
    'slip',
    'incomplete',
    'misunderstanding'
]

np.random.seed(SEED)
random.seed(SEED)

print(f"🎯 Configurações:")
print(f"  - Interações por estudante: {MIN_INTERACTIONS_PER_STUDENT}-{MAX_INTERACTIONS_PER_STUDENT}")
print(f"  - Modelo LLM: {LLM_MODEL}")
print(f"  - Temperatura: {LLM_TEMPERATURE}")
print(f"  - Checkpoint a cada: {CHECKPOINT_INTERVAL} estudantes")
print(f"  - Seed: {SEED}")

🎯 Configurações:
  - Interações por estudante: 30-60
  - Modelo LLM: gpt-4.1-mini
  - Temperatura: 0.7
  - Checkpoint a cada: 10 estudantes
  - Seed: 42


## Funções de Geração com LLM

In [42]:
def generate_response_with_llm(question: Dict, is_correct: bool, attempt: int = 1) -> str:
    """Gera resposta realista usando OpenAI LLM.
    
    Args:
        question: Dicionário com dados da questão
        is_correct: Se a resposta deve estar correta ou incorreta
        attempt: Número da tentativa (para retry)
    
    Returns:
        String com a resposta gerada
    """
    
    question_text = question.get('q', 'Questão desconhecida')
    question_type = question.get('type', 'descriptive')
    
    if question_type == 'multiple_choice':
        if is_correct:
            correct_answer = question.get('ans', 'A')
            return f"Opção {correct_answer}"
        else:
            options = ['A', 'B', 'C', 'D']
            correct = question.get('ans', 'A')
            wrong_options = [o for o in options if o != correct]
            return f"Opção {random.choice(wrong_options)}"
    
    try:
        if is_correct:
            prompt = f"""Você é um estudante de Linux que entendeu bem o conceito. 
            Responda de forma clara e concisa (2-3 frases) a seguinte questão:
            
            {question_text}
            
            Responda como um estudante competente, sem ser muito formal."""
        else:
            prompt = f"""Você é um estudante de Linux que tem uma compreensão errada ou incompleta.
            Responda de forma realista (2-3 frases) a seguinte questão, mas cometendo um erro:
            
            {question_text}
            
            Seu erro pode ser: confundir conceitos, entendimento parcial, ou misconception.
            Responda como um estudante que não entendeu bem."""
        
        response = client.chat.completions.create(
            model=LLM_MODEL,
            messages=[
                {"role": "system", "content": "Você é um estudante respondendo questões sobre Linux."},
                {"role": "user", "content": prompt}
            ],
            temperature=LLM_TEMPERATURE,
            max_tokens=LLM_MAX_TOKENS,
            timeout=LLM_TIMEOUT
        )
        
        return response.choices[0].message.content.strip()
    
    except Exception as e:
        print(f"⚠️  Erro ao chamar LLM (tentativa {attempt}): {str(e)}")
        
        if is_correct:
            return "[Resposta correta - LLM indisponível]"
        else:
            return "[Resposta incorreta - LLM indisponível]"

def calculate_response_probability(student_params: Dict, question_difficulty: float) -> float:
    """Calcula probabilidade de resposta correta baseada em BKT."""
    mastery = student_params.get('mastery_init_level', 0.5)
    guess = student_params.get('guess', 0.15)
    slip = student_params.get('slip', 0.1)
    
    adjusted_mastery = mastery * (1 - question_difficulty * 0.3)
    prob = adjusted_mastery + (1 - adjusted_mastery) * guess - adjusted_mastery * slip
    return max(0, min(1, prob))

def generate_error_explanation(error_type: str, concept_name: str, student_profile: str) -> str:
    """Gera explicação realista para o erro."""
    explanations = {
        'misconception': f"Estudante confundiu o conceito de '{concept_name}' com outro similar. Necessário reforço conceitual.",
        'careless': f"Erro por descuido na execução. Estudante conhece '{concept_name}' mas não prestou atenção.",
        'slip': f"Erro por distração. Estudante sabe '{concept_name}' mas cometeu erro de digitação/lógica.",
        'incomplete': f"Resposta incompleta sobre '{concept_name}'. Faltaram detalhes importantes.",
        'misunderstanding': f"Entendimento errado do enunciado relacionado a '{concept_name}'."
    }
    return explanations.get(error_type, "Erro desconhecido")

print("✅ Funções de geração definidas")

✅ Funções de geração definidas


## Geração de Interações para Todos os Estudantes

In [43]:
def save_checkpoint(interactions: List[Dict], student_idx: int, total_students: int):
    """Salva checkpoint incremental das interações."""
    checkpoint_data = {
        "checkpoint_info": {
            "last_student_processed": student_idx,
            "total_students": total_students,
            "progress_percentage": round((student_idx + 1) / total_students * 100, 2),
            "timestamp": datetime.now().isoformat(),
            "total_interactions": len(interactions)
        },
        "interactions": interactions
    }
    
    with open(CHECKPOINT_FILE, 'w', encoding='utf-8') as f:
        json.dump(checkpoint_data, f, indent=2, ensure_ascii=False)
    
    print(f"  💾 Checkpoint salvo: {student_idx + 1}/{total_students} estudantes ({len(interactions)} interações)")

def load_checkpoint():
    """Carrega checkpoint se existir."""
    if os.path.exists(CHECKPOINT_FILE):
        with open(CHECKPOINT_FILE, 'r', encoding='utf-8') as f:
            data = json.load(f)
        return data['interactions'], data['checkpoint_info']['last_student_processed']
    return [], -1

def generate_interactions(students: List, profiles: Dict, questions: List,
                         min_interactions: int, max_interactions: int,
                         seed: int, resume: bool = True) -> List[Dict]:
    """Gera interações para todos os estudantes com respostas via LLM e salvamento incremental."""
    np.random.seed(seed)
    random.seed(seed)
    
    interactions = []
    interaction_id = 0
    start_idx = 0
    
    if resume:
        interactions, last_processed = load_checkpoint()
        if last_processed >= 0:
            start_idx = last_processed + 1
            interaction_id = len(interactions)
            print(f"📂 Checkpoint encontrado! Retomando do estudante {start_idx + 1}")
            print(f"   Interações já processadas: {len(interactions)}")
    
    total_students = len(students)
    
    for student_idx in range(start_idx, total_students):
        student = students[student_idx]
        
        if (student_idx + 1) % 10 == 0 or student_idx == start_idx:
            print(f"  Processando estudante {student_idx + 1}/{total_students}...")
        
        num_interactions = np.random.randint(min_interactions, max_interactions + 1)
        
        student_id = student['id']
        profile_id = student['profile_id']
        profile = profiles.get(profile_id, {})
        
        current_mastery = student.get('mastery_init_level', 0.5)
        learn_rate = student.get('learn_rate', 0.03)
        slip = student.get('slip', 0.1)
        guess = student.get('guess', 0.15)
        
        student_params = {
            'mastery_init_level': current_mastery,
            'learn_rate': learn_rate,
            'slip': slip,
            'guess': guess
        }
        
        for interaction_num in range(num_interactions):
            if not questions:
                continue
            
            question = random.choice(questions)
            question_id = question.get('id', f'q_{interaction_num}')
            question_type = question.get('type', 'multiple_choice')
            question_difficulty = question.get('score', 2.5) / 5.0
            concept_id = question.get('c_id', 'unknown')
            concept_name = question.get('c_name', 'Conceito')
            
            correct_prob = calculate_response_probability(student_params, question_difficulty)
            is_correct = np.random.random() < correct_prob
            
            response = generate_response_with_llm(question, is_correct)
            
            error_type = None
            error_explanation = None
            if not is_correct:
                error_type = random.choice(ERROR_TYPES)
                error_explanation = generate_error_explanation(
                    error_type,
                    concept_name,
                    profile_id
                )
            
            mastery_before = current_mastery
            if is_correct:
                current_mastery += (1 - current_mastery) * learn_rate
            else:
                current_mastery *= (1 - learn_rate * 0.5)
            
            interaction = {
                'interaction_id': f'int_{interaction_id:06d}',
                'student_id': student_id,
                'question_id': question_id,
                'concept_id': concept_id,
                'question_type': question_type,
                'timestamp': (datetime.now() - timedelta(days=num_interactions-interaction_num)).isoformat(),
                'response': response,
                'is_correct': is_correct,
                'error_type': error_type,
                'error_explanation': error_explanation,
                'mastery_before': round(mastery_before, 4),
                'mastery_after': round(current_mastery, 4),
                'time_spent_seconds': np.random.randint(15, 300)
            }
            
            interactions.append(interaction)
            interaction_id += 1
        
        if (student_idx + 1) % CHECKPOINT_INTERVAL == 0:
            save_checkpoint(interactions, student_idx, total_students)
    
    save_checkpoint(interactions, total_students - 1, total_students)
    
    return interactions

print("🔄 Iniciando geração de interações com LLM...")
print(f"   Total de estudantes: {len(students)}")
print(f"   Interações esperadas: {len(students) * MIN_INTERACTIONS_PER_STUDENT}-{len(students) * MAX_INTERACTIONS_PER_STUDENT}")
print(f"   Salvamento automático a cada {CHECKPOINT_INTERVAL} estudantes")
print()

interactions = generate_interactions(students, profiles, questions,
                                    MIN_INTERACTIONS_PER_STUDENT,
                                    MAX_INTERACTIONS_PER_STUDENT,
                                    SEED,
                                    resume=True)

print(f"\n✅ {len(interactions)} interações geradas com sucesso!")

🔄 Iniciando geração de interações com LLM...
   Total de estudantes: 100
   Interações esperadas: 3000-6000
   Salvamento automático a cada 10 estudantes

  Processando estudante 1/100...
  Processando estudante 10/100...
  💾 Checkpoint salvo: 10/100 estudantes (455 interações)
  Processando estudante 20/100...
  💾 Checkpoint salvo: 20/100 estudantes (875 interações)
  Processando estudante 30/100...
  💾 Checkpoint salvo: 30/100 estudantes (1310 interações)
  Processando estudante 40/100...
  💾 Checkpoint salvo: 40/100 estudantes (1758 interações)
  Processando estudante 50/100...
  💾 Checkpoint salvo: 50/100 estudantes (2178 interações)
  Processando estudante 60/100...
  💾 Checkpoint salvo: 60/100 estudantes (2639 interações)
  Processando estudante 70/100...
  💾 Checkpoint salvo: 70/100 estudantes (3114 interações)
  Processando estudante 80/100...
  💾 Checkpoint salvo: 80/100 estudantes (3558 interações)
  Processando estudante 90/100...
  💾 Checkpoint salvo: 90/100 estudantes (397

## Análise de Qualidade das Interações

In [44]:
def analyze_interactions_quality(interactions: List[Dict], students: List) -> Dict:
    """Analisa qualidade das interações geradas."""
    
    total_interactions = len(interactions)
    correct_interactions = sum(1 for i in interactions if i['is_correct'])
    accuracy = correct_interactions / total_interactions if total_interactions > 0 else 0
    
    error_distribution = defaultdict(int)
    for interaction in interactions:
        if interaction['error_type']:
            error_distribution[interaction['error_type']] += 1
    
    interactions_per_student = defaultdict(int)
    for interaction in interactions:
        interactions_per_student[interaction['student_id']] += 1
    
    empty_responses = sum(1 for i in interactions if not i['response'] or len(i['response'].strip()) == 0)
    
    return {
        'total_interactions': total_interactions,
        'total_students': len(students),
        'avg_interactions_per_student': total_interactions / len(students) if students else 0,
        'correct_interactions': correct_interactions,
        'accuracy': accuracy,
        'error_distribution': dict(error_distribution),
        'interactions_per_student_stats': {
            'min': min(interactions_per_student.values()) if interactions_per_student else 0,
            'max': max(interactions_per_student.values()) if interactions_per_student else 0,
            'mean': np.mean(list(interactions_per_student.values())) if interactions_per_student else 0
        },
        'response_quality': {
            'total_responses': total_interactions,
            'empty_responses': empty_responses,
            'valid_responses': total_interactions - empty_responses,
            'validity_percentage': ((total_interactions - empty_responses) / total_interactions * 100) if total_interactions > 0 else 0
        }
    }

quality_analysis = analyze_interactions_quality(interactions, students)

print("\n📊 Análise de Qualidade das Interações:")
print(f"\n  Estatísticas Gerais:")
print(f"    - Total de interações: {quality_analysis['total_interactions']}")
print(f"    - Total de estudantes: {quality_analysis['total_students']}")
print(f"    - Média por estudante: {quality_analysis['avg_interactions_per_student']:.1f}")
print(f"    - Acurácia geral: {quality_analysis['accuracy']:.1%}")
print(f"\n  Qualidade das Respostas:")
print(f"    - Respostas válidas: {quality_analysis['response_quality']['valid_responses']}/{quality_analysis['response_quality']['total_responses']}")
print(f"    - Taxa de validade: {quality_analysis['response_quality']['validity_percentage']:.1f}%")
print(f"\n  Distribuição de Erros:")
for error_type, count in quality_analysis['error_distribution'].items():
    pct = (count / (quality_analysis['total_interactions'] - quality_analysis['correct_interactions'])) * 100
    print(f"    - {error_type}: {count} ({pct:.1f}%)")


📊 Análise de Qualidade das Interações:

  Estatísticas Gerais:
    - Total de interações: 4450
    - Total de estudantes: 100
    - Média por estudante: 44.5
    - Acurácia geral: 44.4%

  Qualidade das Respostas:
    - Respostas válidas: 4450/4450
    - Taxa de validade: 100.0%

  Distribuição de Erros:
    - misconception: 490 (19.8%)
    - slip: 513 (20.7%)
    - careless: 462 (18.7%)
    - misunderstanding: 513 (20.7%)
    - incomplete: 497 (20.1%)


## Salvamento das Interações

In [45]:
# Criar estrutura completa com metadados
output_data = {
    "metadata": {
        "description": "Conjunto de interações simuladas com respostas geradas por LLM para estudantes SINKT",
        "version": "2.0.0",
        "created_at": datetime.now().isoformat(),
        "llm_model": LLM_MODEL,
        "llm_temperature": LLM_TEMPERATURE,
        "total_interactions": len(interactions),
        "total_students": len(students),
        "avg_interactions_per_student": quality_analysis['avg_interactions_per_student'],
        "accuracy": quality_analysis['accuracy'],
        "error_types": ERROR_TYPES,
        "quality_metrics": quality_analysis
    },
    "interactions": interactions
}

output_file = 'data/output/interactions.json'
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(output_data, f, indent=2, ensure_ascii=False)

if os.path.exists(CHECKPOINT_FILE):
    os.remove(CHECKPOINT_FILE)
    print(f"🗑️  Checkpoint removido (processamento completo)")

print(f"\n✅ Interações salvas em: {output_file}")
print(f"📦 Total de interações: {len(interactions)}")
print(f"💾 Tamanho do arquivo: {os.path.getsize(output_file) / (1024*1024):.2f} MB")

🗑️  Checkpoint removido (processamento completo)

✅ Interações salvas em: data/output/interactions.json
📦 Total de interações: 4450
💾 Tamanho do arquivo: 2.61 MB


## Exemplos de Interações Geradas

In [46]:
print("\n📋 Exemplos de Interações Geradas:\n")
print("="*80)

# Mostrar 3 exemplos
for i, interaction in enumerate(interactions[:3]):
    print(f"\nExemplo {i+1}:")
    print(f"  ID: {interaction['interaction_id']}")
    print(f"  Estudante: {interaction['student_id']}")
    print(f"  Questão: {interaction['question_id']}")
    print(f"  Tipo: {interaction['question_type']}")
    print(f"  Resposta: {interaction['response'][:100]}..." if len(interaction['response']) > 100 else f"  Resposta: {interaction['response']}")
    print(f"  Correta: {'✓ SIM' if interaction['is_correct'] else '✗ NÃO'}")
    if interaction['error_type']:
        print(f"  Tipo de Erro: {interaction['error_type']}")
        print(f"  Explicação: {interaction['error_explanation']}")
    print(f"  Domínio Antes: {interaction['mastery_before']:.3f}")
    print(f"  Domínio Depois: {interaction['mastery_after']:.3f}")
    print(f"  Tempo: {interaction['time_spent_seconds']}s")
    print("-" * 80)


📋 Exemplos de Interações Geradas:


Exemplo 1:
  ID: int_000000
  Estudante: student_0000
  Questão: concept_262_q4
  Tipo: descriptive
  Resposta: A Hierarquia de Diretórios é uma lista dos arquivos que estão instalados no sistema, e a Estrutura d...
  Correta: ✗ NÃO
  Tipo de Erro: misconception
  Explicação: Estudante confundiu o conceito de 'Hierarquia de Diretórios' com outro similar. Necessário reforço conceitual.
  Domínio Antes: 0.529
  Domínio Depois: 0.519
  Tempo: 285s
--------------------------------------------------------------------------------

Exemplo 2:
  ID: int_000001
  Estudante: student_0000
  Questão: concept_008_q1
  Tipo: multiple_choice
  Resposta: Opção D
  Correta: ✗ NÃO
  Tipo de Erro: slip
  Explicação: Erro por distração. Estudante sabe '-C' mas cometeu erro de digitação/lógica.
  Domínio Antes: 0.519
  Domínio Depois: 0.508
  Tempo: 203s
--------------------------------------------------------------------------------

Exemplo 3:
  ID: int_000002
  Estud

## Resumo da Execução

In [47]:
print("\n" + "="*70)
print("🎉 GERAÇÃO DE INTERAÇÕES COM LLM CONCLUÍDA COM SUCESSO!")
print("="*70)
print(f"\n📁 Arquivo gerado:")
print(f"  - {output_file}")
print(f"\n📊 Resumo:")
print(f"  - Total de interações: {len(interactions)}")
print(f"  - Estudantes: {len(students)}")
print(f"  - Média por estudante: {quality_analysis['avg_interactions_per_student']:.1f}")
print(f"  - Acurácia: {quality_analysis['accuracy']:.1%}")
print(f"  - Modelo LLM: {LLM_MODEL}")
print(f"  - Respostas válidas: {quality_analysis['response_quality']['validity_percentage']:.1f}%")
print(f"\n✅ Próximo passo: Execute o notebook '04_analise_metricas.ipynb'")
print("="*70)


🎉 GERAÇÃO DE INTERAÇÕES COM LLM CONCLUÍDA COM SUCESSO!

📁 Arquivo gerado:
  - data/output/interactions.json

📊 Resumo:
  - Total de interações: 4450
  - Estudantes: 100
  - Média por estudante: 44.5
  - Acurácia: 44.4%
  - Modelo LLM: gpt-4.1-mini
  - Respostas válidas: 100.0%

✅ Próximo passo: Execute o notebook '04_analise_metricas.ipynb'
